# Group Analysis - FOOOFed EEG Analysis: Task
    
Applying FOOOF to task based EEG data, and comparing between young and old groups. 

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

%load_ext autoreload

In [2]:
from os.path import join as pjoin
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import pearsonr

import patsy
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm

In [3]:
# Import custom code for this analysis
%autoreload 2
from fio import *
from plts import *
from utils import *

# Settings

In [4]:
# Set path to load results from
res_path = '/Users/tom/Documents/Research/1-Projects/fooof/2-Data/Results/'

# Set indices to separate groups
from settings import YNG_INDS, OLD_INDS

# Set average function to use
avg_func = np.nanmean
#avg_func = np.nanmedian

# Wether to save out plots or not
save_fig = False

print('Number of young subjects:  ', len(YNG_INDS))
print('Number of  old  subjects:  ', len(OLD_INDS))

Number of young subjects:   17
Number of  old  subjects:   14


In [5]:
# Data settings
srate = 512
tmin, tmax = -0.85, 1.1
times = np.arange(tmin, tmax, 1/srate)
seg_times = [(-0.85, -0.35), (0.1, 0.6), (0.5, 1.0)]
n_subjs = 31

### Check dropped trials

In [6]:
# Load dropped trials & components
dropped_trials = np.load(pjoin(res_path, 'Group', 'dropped_trials.npy'))
dropped_components = np.load(pjoin(res_path, 'Group', 'dropped_components.npy'))

# Check dropped trials for each subject
print('SubNum \t\t # Dropped Trials \t # Dropped Components')
for ind, trials, components in zip(range(n_subjs), dropped_trials, dropped_components):
    temp_trials = trials[trials < 999.]
    temp_comps = components[components < 999.]
    print(ind, '\t\t', len(temp_trials), '\t\t\t', len(temp_comps))

SubNum 		 # Dropped Trials 	 # Dropped Components
0 		 7 			 4
1 		 21 			 5
2 		 1 			 0
3 		 12 			 2
4 		 18 			 3
5 		 0 			 8
6 		 24 			 6
7 		 7 			 0
8 		 3 			 5
9 		 4 			 4
10 		 5 			 4
11 		 7 			 5
12 		 13 			 7
13 		 0 			 3
14 		 2 			 4
15 		 1 			 5
16 		 7 			 4
17 		 8 			 0
18 		 1 			 3
19 		 137 			 3
20 		 31 			 6
21 		 15 			 5
22 		 0 			 3
23 		 0 			 4
24 		 2 			 4
25 		 22 			 0
26 		 4 			 4
27 		 24 			 5
28 		 43 			 4
29 		 48 			 5
30 		 0 			 4


## Group FOOOFing - Trial Averaged Data

Notes:
- `all_alphas` & `all_exps` have the shape [n_loads, n_subjs, n_times]

### Load Data

In [7]:
# Load behavioural data,
behav_dat = pd.read_csv(pjoin(res_path, 'Behav', 'neural_aging_data_behaviour.csv'))

In [8]:
# Convert data types
behav_dat['Age'] = behav_dat['Age'].astype('str')
behav_dat['Load'] = behav_dat['Load'].astype('str')

In [9]:
# Calculate average behaviour across loads
avg_behav = behav_dat.groupby('SubjID').mean()

In [10]:
# CHEATING
#for ind in [28, 59, 90]:
#    behav_dat.loc[ind, "d'"] = 2

In [11]:
# Load and extract FOOOF data
all_exps, all_alphas = load_fooof_task(res_path, 'Contra')

In [12]:
# Replace NaN with row mean
# import numpy.ma as ma
# out = []
# for mat in all_alphas:
#     mat = mat.T
#     out.append(np.where(np.isnan(mat), ma.array(mat, mask=np.isnan(mat)).mean(axis=0), mat).T)
# all_alphas = np.stack(out)

# Replace nan with global mean
#all_alphas[np.isnan(all_alphas)] = np.nanmean(all_alphas)

In [13]:
# Load canonical alpha analysis
canonical_group = np.load(pjoin(res_path, 'Group', 'canonical_group.npy'))

In [14]:
# Average across analytic alpha measures to get canonical alpha measure
seg_masks = []
for seg in seg_times:
    seg_masks.append(np.logical_and(times >= seg[0], times <= seg[1]))

canalpha = np.zeros_like(all_alphas)
for subi, subj_dat in enumerate(canonical_group):
    for lodi in range(3):
        for segi, mask in enumerate(seg_masks):
            canalpha[lodi, subi, segi] = np.mean(subj_dat[lodi, mask])

In [15]:
# Number of missing FOOOFed alphas
sum(sum(sum(np.isnan(all_alphas))))

10

In [16]:
# # TESTS
#all_alphas[:, 1, :] = np.nan
#canalpha [:, 1, :] = np.nan
# all_alphas[:, 8, :] = np.nan
# canalpha [:, 8, :] = np.nan

In [17]:
# CHEATING: DROP OUT SUBJECTS
#behav_dat[behav_dat['SubjID'] == 8] = np.nan
#behav_dat[behav_dat['SubjID'] == 1] = np.nan

## Check Behaviour & Trial EPhys Corrs - Baseline Measures

In [18]:
# Predict behaviour from 'absolute' FOOOF trial output measures
#   Note: as defined this model has three measures per subject, one for each load

# Settings
t_ind = 0

# Build the dataframe
df = pd.DataFrame()
df['exponent'] = all_exps[:, :, t_ind].flatten()

#df['alpha'] = all_alphas[:, :, t_ind].flatten()
df['alpha'] = canalpha[:, :, t_ind].flatten()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values

# Specify the model
#outcome, predictors = patsy.dmatrices("behav ~ exponent + alpha + age", df)
#outcome, predictors = patsy.dmatrices("behav ~ exponent * alpha * age", df)
outcome, predictors = patsy.dmatrices("behav ~ age + alpha + exponent + alpha:age + exponent:age", df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()

# Check out the results
#print(res.summary())

In [19]:
# # Predict behaviour from 'absolute' FOOOF trial output measures, separately for each load

# # Settings
# t_ind = 2
# l_ind = 2

# # Build the dataframe
# df = pd.DataFrame()
# df['exponent'] = all_exps[l_ind, :, t_ind].flatten()
# df['alpha'] = all_alphas[l_ind, :, t_ind].flatten()

# df['behav'] = behav_dat[behav_dat['Load'] == l_ind+1]["d'"].values
# df['age'] = behav_dat[behav_dat['Load'] == l_ind+1]["Age"].values

# # Specift the model
# #outcome, predictors = patsy.dmatrices("behav ~ exponent + age", df)
# outcome, predictors = patsy.dmatrices("behav ~ exponent + alpha + age", df)
# #outcome, predictors = patsy.dmatrices("behav ~ exponent * alpha * age", df)
# #outcome, predictors = patsy.dmatrices("behav ~ age + alpha + exponent + alpha:age + exponent:age", df)
# mod = sm.OLS(outcome, predictors)
# res = mod.fit()

# # Check out the results
# #print(res.summary())

In [20]:
# # Predict from absolute FOOOF results, split up by group

# # Settings
# #group = 'young' # 'young', 'old'
# group = 'old' 
# t_ind = 2

# inds = YNG_INDS if group == 'young' else OLD_INDS
# age_gr = 1 if group == 'young' else 2

# # Build the dataframe
# df = pd.DataFrame()
# df['exponent'] = all_exps[:, inds, t_ind].flatten()

# df['alpha'] = all_alphas[:, inds, t_ind].flatten()
# #df['alpha'] = canalpha[:, inds, t_ind].flatten()

# df['behav'] = behav_dat[behav_dat.Age == age_gr]["d'"].values

# # 
# outcome, predictors = patsy.dmatrices("behav ~ exponent * alpha", df)
# mod = sm.OLS(outcome, predictors)
# res = mod.fit()

# # Check out the results
# print(res.summary())

In [21]:
# # Check relation between exponent and alpha
# print(pearsonr(df.exponent, df.alpha))
# #plt.plot(df.exponent, df.alpha, '.')

# print(pearsonr(df.behav, df.alpha))
# #plt.plot(df.behav, df.alpha, '.')

# print(pearsonr(df.behav, df.exponent))
# plt.plot(df.behav, df.exponent, '.')

## Difference Measures

Predict behaviour output from evoked responses of alpha and aperiodic. 

In [22]:
#
sl_st_yng = np.vstack([all_exps[0, YNG_INDS, :], all_exps[1, YNG_INDS, :], all_exps[2, YNG_INDS, :]])
sl_st_old = np.vstack([all_exps[0, OLD_INDS, :], all_exps[1, OLD_INDS, :], all_exps[2, OLD_INDS, :]])

#
al_st_yng = np.vstack([all_alphas[0, YNG_INDS, :], all_alphas[1, YNG_INDS, :], all_alphas[2, YNG_INDS, :]])
al_st_old = np.vstack([all_alphas[0, OLD_INDS, :], all_alphas[1, OLD_INDS, :], all_alphas[2, OLD_INDS, :]])

#
canal_st_yng = np.vstack([canalpha[0, YNG_INDS, :], canalpha[1, YNG_INDS, :], canalpha[2, YNG_INDS, :]])
canal_st_old = np.vstack([canalpha[0, OLD_INDS, :], canalpha[1, OLD_INDS, :], canalpha[2, OLD_INDS, :]])

In [23]:
# Calculate difference measures
i1, i2 = 2, 0

sl_dif_yng = calc_diff(sl_st_yng, i1, i2)
sl_dif_old = calc_diff(sl_st_old, i1, i2)

al_dif_yng = calc_diff(al_st_yng, i1, i2)
al_dif_old = calc_diff(al_st_old, i1, i2)

canal_dif_yng = calc_diff(canal_st_yng, i1, i2)
canal_dif_old = calc_diff(canal_st_old, i1, i2)

In [24]:
# Check correlations of difference measures with behaviour
print_stat('Sl-Yng', *pearsonr(sl_dif_yng, behav_dat[behav_dat.Age == '1']["d'"]))
print_stat('Sl-Old', *pearsonr(sl_dif_old, behav_dat[behav_dat.Age == '2']["d'"]))
print('')
print_stat('Al-Yng', *pearsonr(al_dif_yng, behav_dat[behav_dat.Age == '1']["d'"]))
print_stat('Al-Old', *pearsonr(al_dif_old, behav_dat[behav_dat.Age == '2']["d'"]))
print('')
print_stat('Al-Yng', *pearsonr(canal_dif_yng, behav_dat[behav_dat.Age == '1']["d'"]))
print_stat('Al-Old', *pearsonr(canal_dif_old, behav_dat[behav_dat.Age == '2']["d'"]))

Sl-Yng: 	  0.3171 	0.0234
Sl-Old: 	 -0.1065 	0.5021

Al-Yng: 	   nan 	1.0000
Al-Old: 	   nan 	1.0000

Al-Yng: 	 -0.1788 	0.2092
Al-Old: 	  0.2539 	0.1046


## Base Model: Age & Load

In [25]:
df = pd.DataFrame()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values
df['load'] = behav_dat['Load'].values
 
outcome, predictors = patsy.dmatrices("behav ~ load + age", df)
mod_base = sm.OLS(outcome, predictors).fit()

print(mod_base.summary())

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     8.761
Date:                Wed, 20 Feb 2019   Prob (F-statistic):           3.75e-05
Time:                        15:24:39   Log-Likelihood:                -118.31
No. Observations:                  93   AIC:                             244.6
Df Residuals:                      89   BIC:                             254.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.0845      0.179     22.822      0.0

### CDA Model

In [26]:
df = pd.DataFrame()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values
df['load'] = behav_dat['Load'].values
df['cda'] = behav_dat['CDA'].values
 
outcome, predictors = patsy.dmatrices("behav ~ load + cda + age", df)
mod_base = sm.OLS(outcome, predictors).fit()

print(mod_base.summary())

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     6.556
Date:                Wed, 20 Feb 2019   Prob (F-statistic):           0.000115
Time:                        15:24:39   Log-Likelihood:                -118.21
No. Observations:                  93   AIC:                             246.4
Df Residuals:                      88   BIC:                             259.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.1246      0.203     20.327      0.0

### Predicting from Canonical Alpha

In [27]:
df = pd.DataFrame()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values
df['load'] = behav_dat['Load'].values

df['al_dif'] = np.concatenate([canal_dif_yng, canal_dif_old])
 
outcome, predictors = patsy.dmatrices("behav ~ load + al_dif * age", df)
mod_alpha_can = sm.OLS(outcome, predictors).fit()

print(mod_alpha_can.summary())

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                     7.268
Date:                Wed, 20 Feb 2019   Prob (F-statistic):           9.93e-06
Time:                        15:24:39   Log-Likelihood:                -114.11
No. Observations:                  93   AIC:                             240.2
Df Residuals:                      87   BIC:                             255.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.0041      0.176     

## Predicting from FOOOF: Alpha

In [28]:
df = pd.DataFrame()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values
df['load'] = behav_dat['Load'].values

df['al_dif'] = np.concatenate([al_dif_yng, al_dif_old])
 
outcome, predictors = patsy.dmatrices("behav ~ load + al_dif * age", df)
mod_alpha_foo = sm.OLS(outcome, predictors).fit()

print(mod_alpha_foo.summary())

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.330
Model:                            OLS   Adj. R-squared:                  0.289
Method:                 Least Squares   F-statistic:                     7.993
Date:                Wed, 20 Feb 2019   Prob (F-statistic):           3.66e-06
Time:                        15:24:39   Log-Likelihood:                -105.68
No. Observations:                  87   AIC:                             223.4
Df Residuals:                      81   BIC:                             238.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.0393      0.183     

## Predicting from FOOOF: Aperiodic

In [29]:
df = pd.DataFrame()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values
df['load'] = behav_dat['Load'].values

df['sl_dif'] = np.concatenate([sl_dif_yng, sl_dif_old])

outcome, predictors = patsy.dmatrices("behav ~ load + age * sl_dif", df)
mod_exponent_foo = sm.OLS(outcome, predictors).fit()

print(mod_exponent_foo.summary())

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.214
Method:                 Least Squares   F-statistic:                     6.007
Date:                Wed, 20 Feb 2019   Prob (F-statistic):           8.04e-05
Time:                        15:24:39   Log-Likelihood:                -116.55
No. Observations:                  93   AIC:                             245.1
Df Residuals:                      87   BIC:                             260.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.1944      0.188     

### Predicting from FOOOF model with Alpha & Aperiodic

In [30]:
df = pd.DataFrame()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values
df['load'] = behav_dat['Load'].values

df['sl_dif'] = np.concatenate([sl_dif_yng, sl_dif_old])
df['al_dif'] = np.concatenate([al_dif_yng, al_dif_old])

outcome, predictors = patsy.dmatrices("behav ~ load + age * al_dif + sl_dif", df)
mod_all_foo = sm.OLS(outcome, predictors).fit()

print(mod_all_foo.summary())

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     6.733
Date:                Wed, 20 Feb 2019   Prob (F-statistic):           8.45e-06
Time:                        15:24:39   Log-Likelihood:                -105.34
No. Observations:                  87   AIC:                             224.7
Df Residuals:                      80   BIC:                             241.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.0609      0.186     

## Model Comparisons

In [31]:
# Compare model with FOOOF alpha vs. model with canonical alpha
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('Canonical alpha vs. Base Model')
    print(anova_lm(mod_base, mod_alpha_can))
    print('\n')
    print('FOOOFed alpha vs. Base Model')
    print(anova_lm(mod_base, mod_alpha_foo))
    print('\n')
    print('Canonical alpha vs. FOOOFed Alpha')
    print(anova_lm(mod_alpha_can, mod_alpha_foo))

Canonical alpha vs. Base Model
   df_resid        ssr  df_diff   ss_diff         F    Pr(>F)
0      88.0  69.194025      0.0       NaN       NaN       NaN
1      87.0  63.351513      1.0  5.842512  8.023463  0.005737


FOOOFed alpha vs. Base Model
   df_resid        ssr  df_diff   ss_diff        F    Pr(>F)
0      88.0  69.194025      0.0       NaN      NaN       NaN
1      81.0  57.823785      7.0  11.37024  2.27536  0.036273


Canonical alpha vs. FOOOFed Alpha
   df_resid        ssr  df_diff   ss_diff         F    Pr(>F)
0      87.0  63.351513      0.0       NaN       NaN       NaN
1      81.0  57.823785      6.0  5.527728  1.290547  0.270927


In [32]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('FOOOF aperiodic vs. Base Model')
    print(anova_lm(mod_base, mod_exponent_foo))    

FOOOF aperiodic vs. Base Model
   df_resid        ssr  df_diff   ss_diff         F    Pr(>F)
0      88.0  69.194025      0.0       NaN       NaN       NaN
1      87.0  66.764532      1.0  2.429493  3.165841  0.078686


In [33]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('FOOOF Full vs. Canonical Alpha')
    print(anova_lm(mod_alpha_can, mod_all_foo))

FOOOF Full vs. Canonical Alpha
   df_resid        ssr  df_diff   ss_diff         F    Pr(>F)
0      87.0  63.351513      0.0       NaN       NaN       NaN
1      80.0  57.377602      7.0  5.973911  1.189894  0.318107


In [34]:
# Compare FOOOF model with exponent to restricted model with just alpha (from FOOOF)
lm_val, p_val, df_diff = mod_all_foo.compare_lm_test(mod_alpha_foo)
print_stat('Model comp', lm_val, p_val)

Model comp: 	  0.6713 	0.4126


## Correlations

????

In [36]:
# Check correlation between exponent & d', and alpha & d' - for a specific load and time point

# Set which load and time point to use
l_ind = 0
t_ind = 0

print_stat("Sl-Lo-d'", *pearsonr(all_exps[l_ind, :, t_ind],
                                 behav_dat[behav_dat.Load == l_ind + 1]["d'"]))
print_stat("Al-Lo-d'", *pearsonr(all_alphas[l_ind, :, t_ind],
                                 behav_dat[behav_dat.Load == l_ind + 1]["d'"]))

ValueError: operands could not be broadcast together with shapes (31,) (0,) 

In [37]:
t_ind = 2

print_stat("Sl-d'", *pearsonr(all_exps[:, :, t_ind].flatten(), behav_dat["d'"].values))
print_stat("Al-d'", *pearsonr(all_alphas[:, :, t_ind].flatten(), behav_dat["d'"].values))

Sl-d': 	 -0.2382 	0.0215
Al-d': 	   nan 	1.0000


In [38]:
# Average d' with average measures

t_ind = 2

print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_exps[:, :, t_ind], 0)))
print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_alphas[:, :, t_ind], 0)))

XX: 	 -0.2947 	0.1075
XX: 	   nan 	1.0000


In [39]:
# Average d' with average measures

t_ind = 2

print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_exps[:, :, 2] - all_exps[:, :, 0], 0)))
print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_alphas[:, :, 2] - all_alphas[:, :, 0], 0)))

XX: 	 -0.1450 	0.4365
XX: 	   nan 	1.0000


In [40]:
# ind = 2

# print_stat('Yng-Sl', *pearsonr(sl_st_yng[:, ind], behav_dat[behav_dat.Age == 1]["d'"]))
# print_stat('Old-Sl', *pearsonr(sl_st_old[:, ind], behav_dat[behav_dat.Age == 2]["d'"]))

# print('')

# print_stat('Yng-Al', *pearsonr(al_st_yng[:, ind], behav_dat[behav_dat.Age == 1]["d'"]))
# print_stat('Old-Al', *pearsonr(al_st_old[:, ind], behav_dat[behav_dat.Age == 2]["d'"]))

# print('')

# print_stat('Yng-CanAl', *pearsonr(canal_st_yng[:, ind], behav_dat[behav_dat.Age == 1]["d'"]))
# print_stat('Old-CanAl', *pearsonr(canal_st_old[:, ind], behav_dat[behav_dat.Age == 2]["d'"]))

In [41]:
# Average across loads
sl_resps = np.diff(np.mean(all_exps, 0))
al_resps = np.diff(np.mean(all_alphas, 0))

# # Take specific load
# load_ind = 2
# sl_resps = all_exps[load_ind, :, :]
# al_resps = all_alphas[load_ind, :, :]

In [42]:
# Correlations between reactivity measures
print_stat('SLR & ALR - 1', *pearsonr(sl_resps[:, 0], al_resps[:, 0]))
print_stat('SLR & ALR - 2', *pearsonr(sl_resps[:, 1], al_resps[:, 1]))

SLR & ALR - 1: 	   nan 	1.0000
SLR & ALR - 2: 	   nan 	1.0000


In [43]:
edat = sl_resps; ind = 0;

print_stat('\tALL', *pearsonr(avg_behav["d'"], al_resps[:, 0]))
print_stat('\tYNG', *pearsonr(avg_behav[avg_behav['Age'] == 1]["d'"], edat[YNG_INDS, ind]))
print_stat('\tOLD', *pearsonr(avg_behav[avg_behav['Age'] == 2]["d'"], edat[OLD_INDS, ind]))

	ALL: 	   nan 	1.0000


KeyError: 'Age'

In [44]:
edat = sl_resps

print_stat('\tALL', *pearsonr(avg_behav["d'"], al_resps[:, 0]))
print_stat('\tYNG', *pearsonr(avg_behav[avg_behav['Age'] == 1]["d'"], np.mean(edat[YNG_INDS], 1)))
print_stat('\tOLD', *pearsonr(avg_behav[avg_behav['Age'] == 2]["d'"], np.mean(edat[OLD_INDS], 1)))

	ALL: 	   nan 	1.0000


KeyError: 'Age'

In [45]:
print(pearsonr(avg_behav["CDA"], al_resps[:, 0]))
print(pearsonr(avg_behav["CDA"], sl_resps[:, 0]))

(nan, 1.0)
(-0.3622129765020239, 0.04523951010175264)


In [46]:
print(pearsonr(sl_dif_yng, al_dif_yng))
print(pearsonr(sl_dif_old, al_dif_old))

(nan, 1.0)
(nan, 1.0)


## Miscellaneous Crap

In [47]:
# # Build the dataframe

# t_ind = 2
# l_ind = 0 

# df = pd.DataFrame()

# df['behav'] = behav_dat[behav_dat['Load'] == l_ind+1]["d'"].values
# df['age'] = behav_dat[behav_dat['Load'] == l_ind+1]["Age"].values

# df['sl_dif'] = np.concatenate([sl_dif_yng, sl_dif_old])
# df['al_dif'] = np.concatenate([al_dif_yng, al_dif_old])

# # 
# outcome, predictors = patsy.dmatrices("behav ~ sl_dif + al_dif + age", df)
# #outcome, predictors = patsy.dmatrices("behav ~ age * sl_dif", df)
# mod = sm.OLS(outcome, predictors)
# res = mod.fit()

# # Check out the results
# print(res.summary())

In [48]:
#
al_ = np.mean(all_exps[:, YNG_INDS, :], 0)

sl_st_yng = np.vstack([all_exps[0, YNG_INDS, :], all_exps[1, YNG_INDS, :], all_exps[2, YNG_INDS, :]])
sl_st_old = np.vstack([all_exps[0, OLD_INDS, :], all_exps[1, OLD_INDS, :], all_exps[2, OLD_INDS, :]])

#
al_st_yng = np.vstack([all_alphas[0, YNG_INDS, :], all_alphas[1, YNG_INDS, :], all_alphas[2, YNG_INDS, :]])
al_st_old = np.vstack([all_alphas[0, OLD_INDS, :], all_alphas[1, OLD_INDS, :], all_alphas[2, OLD_INDS, :]])

In [49]:
# Get the change in behavioural scores between highes & lowest loads
delta_behavs = []
temp_df = behav_dat[behav_dat.Age == 1]
for subj_id in set(temp_df.SubjID.values):
    temp = behav_dat[behav_dat.SubjID == subj_id]
    delta_behavs.append(temp[temp.Load == 3]["d'"].values[0] - temp[temp.Load == 1]["d'"].values[0])
delta_behavs = np.array(delta_behavs)

In [50]:
#behav_dat.Age == 2

In [51]:
# Get the change in physiological scores between highes & lowest loads
temp_dat = all_exps
temp = temp_dat[2, :, :] - temp_dat[0, :, :]
temp[np.isnan(temp)] = 0
delta_phys = temp[:, 2] - temp[:, 0]

In [52]:
print('ALL:', pearsonr(delta_behavs, delta_phys[OLD_INDS]))
plt.scatter(delta_behavs, delta_phys[OLD_INDS])

/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3005: RuntimeWarning: Mean of empty slice.
  mx = x.mean()


ValueError: operands could not be broadcast together with shapes (0,) (14,) 

In [53]:
print('ALL:', pearsonr(delta_behavs, delta_phys[YNG_INDS]))
plt.scatter(delta_behavs, delta_phys[YNG_INDS])

ValueError: operands could not be broadcast together with shapes (0,) (17,) 

In [54]:
plt.scatter(delta_behavs[OLD_INDS], delta_phys[OLD_INDS], label='OLD')
print('OLD:', pearsonr(delta_behavs[OLD_INDS], delta_phys[OLD_INDS]))
plt.scatter(delta_behavs[YNG_INDS], delta_phys[YNG_INDS], label='YNG')
print('YNG:', pearsonr(delta_behavs[YNG_INDS], delta_phys[YNG_INDS]))
plt.legend()
plt.xlabel('Delta Behav');
plt.ylabel('Delta Physiology');

IndexError: index 0 is out of bounds for axis 1 with size 0

In [55]:
#cur_dat = canalpha
#cur_dat = all_exps
cur_dat = all_alphas

# Get phys-responsiveness, as late minus pre
temp = cur_dat[:, :, 2] - cur_dat[:, :, 1]
# Or - just take late
#temp = cur_dat[:, :, 2]

# Replace any nans as zeros
temp[np.isnan(temp)] = 0

# Get delta physiology as high load minus low load
delta_phys = temp[2, :] - temp[0, :]

In [56]:
print('ALL:', pearsonr(delta_behavs, delta_phys))
print('OLD:', pearsonr(delta_behavs[OLD_INDS], delta_phys[OLD_INDS]))
print('YNG:', pearsonr(delta_behavs[YNG_INDS], delta_phys[YNG_INDS]))

plt.scatter(delta_behavs[OLD_INDS], delta_phys[OLD_INDS], label='OLD')
plt.scatter(delta_behavs[YNG_INDS], delta_phys[YNG_INDS], label='YNG')

#plt.ylim([-0.0000005, 0.0000005])

plt.legend()
plt.xlabel('Delta Behav');
plt.ylabel('Delta Physiology');

ValueError: operands could not be broadcast together with shapes (0,) (31,) 

In [57]:
#
sl_st_yng = np.vstack([all_exps[0, YNG_INDS, :], all_exps[1, YNG_INDS, :], all_exps[2, YNG_INDS, :]])
sl_st_old = np.vstack([all_exps[0, OLD_INDS, :], all_exps[1, OLD_INDS, :], all_exps[2, OLD_INDS, :]])

#
al_st_yng = np.vstack([all_alphas[0, YNG_INDS, :], all_alphas[1, YNG_INDS, :], all_alphas[2, YNG_INDS, :]])
al_st_old = np.vstack([all_alphas[0, OLD_INDS, :], all_alphas[1, OLD_INDS, :], all_alphas[2, OLD_INDS, :]])

#
canal_st_yng = np.vstack([canalpha[0, YNG_INDS, :], canalpha[1, YNG_INDS, :], canalpha[2, YNG_INDS, :]])
canal_st_old = np.vstack([canalpha[0, OLD_INDS, :], canalpha[1, OLD_INDS, :], canalpha[2, OLD_INDS, :]])

In [58]:
for si, dd in zip(range(31), sum(np.isnan(all_alphas), 0).sum(1)):
    print(si, '\t', dd)

0 	 0
1 	 5
2 	 0
3 	 1
4 	 0
5 	 0
6 	 0
7 	 0
8 	 3
9 	 0
10 	 0
11 	 0
12 	 0
13 	 0
14 	 0
15 	 0
16 	 0
17 	 0
18 	 0
19 	 0
20 	 0
21 	 0
22 	 0
23 	 0
24 	 0
25 	 0
26 	 0
27 	 0
28 	 0
29 	 1
30 	 0
